In [10]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import PIL.Image as image
import numpy as np
import pandas as pd
import pyecharts
from pyecharts.charts import Map
from pyecharts import options as opts

In [4]:
html = 'https://voice.baidu.com/act/newpneumonia/newpneumonia/?from=osari_aladin_banner'
chrome_driver = r'D:\anaconda\envs\pytry\Lib\site-packages\selenium\chromedriver.exe'
browser = webdriver.Chrome(executable_path=chrome_driver)
browser.set_window_size(1000, 800)  # 设置窗口大小
try:
    browser.get(html)
    # eles = browser.find_elements_by_class_name('Common_1-1-321_3lDRV2')  # 找到该按钮
    # ele = eles[0]
    # browser.execute_script("arguments[0].scrollIntoView();", ele)  # 滚动至该按钮可见位置

    """
    browser.find_element_by_class_name('li-page.iconfont.icon-icon_pagedown')
    括号里填的内容，如果有空格由 . 代替
    """

    # browser.execute_script("arguments[0].scrollIntoView();", ele)  # 滚动至该按钮可见位置，要不然无法模拟点击
    buttons = browser.find_elements_by_class_name(
        'Common_1-1-321_3lDRV2')  # 查找该按钮
    # ActionChains(browser).move_to_element_with_offset(button, 500, 10).click().perform()
    for button in buttons:
        button.click()
    # time.sleep(1)  # 停止两秒
    with open('source_page.html', 'w', encoding='utf-8') as fp:
        fp.write(browser.page_source)
    print('source_page 爬取完成')
finally:
    browser.close()

source_page 爬取完成


In [5]:
texts = []
with open('source_page.html', 'r', encoding='utf-8')as fp:
    file = fp.read()
    soup1 = BeautifulSoup(file, 'lxml')
    soup = BeautifulSoup(soup1.prettify(), 'lxml')
    temp = soup.find_all(attrs={'class': 'VirusTable_1-1-321_3m6Ybq'})
    for th in temp:
        texts.append(th.text.replace('\n', ' ').split(' '))
    for text in texts:
        for i in range(text.count('')):
            text.remove('')
texts

[['台湾', '38', '2299', '16891', '13742', '850'],
 ['浙江', '1', '504', '2003', '1498', '1'],
 ['陕西', '86', '369', '1078', '706', '3'],
 ['广西', '10', '236', '568', '330', '2'],
 ['内蒙古', '0', '231', '1186', '954', '1'],
 ['香港', '5', '153', '12555', '12189', '213'],
 ['上海', '15', '138', '3019', '2874', '7'],
 ['云南', '6', '96', '1799', '1701', '2'],
 ['广东', '6', '88', '3419', '3323', '8'],
 ['河南', '0', '32', '1640', '1586', '22'],
 ['山东', '1', '29', '1038', '1002', '7'],
 ['天津', '9', '29', '554', '522', '3'],
 ['福建', '1', '28', '1347', '1318', '1'],
 ['四川', '1', '28', '1306', '1275', '3'],
 ['北京', '0', '14', '1205', '1182', '9'],
 ['湖南', '4', '13', '1212', '1195', '4'],
 ['甘肃', '0', '12', '356', '342', '2'],
 ['黑龙江', '0', '10', '2035', '2012', '13'],
 ['辽宁', '0', '9', '793', '782', '2'],
 ['吉林', '0', '7', '589', '579', '3'],
 ['江苏', '0', '5', '1624', '1619', '0'],
 ['重庆', '0', '2', '610', '602', '6'],
 ['安徽', '0', '1', '1009', '1002', '6'],
 ['湖北', '0', '0', '68316', '63804', '4512'],
 ['河北',

In [14]:
arr = np.array(texts)
province = list(arr[:, 0])
now_exits = list(arr[:, 2])
data = [list(z)for z in zip(province, now_exits)]
data

[['台湾', '2299'],
 ['浙江', '504'],
 ['陕西', '369'],
 ['广西', '236'],
 ['内蒙古', '231'],
 ['香港', '153'],
 ['上海', '138'],
 ['云南', '96'],
 ['广东', '88'],
 ['河南', '32'],
 ['山东', '29'],
 ['天津', '29'],
 ['福建', '28'],
 ['四川', '28'],
 ['北京', '14'],
 ['湖南', '13'],
 ['甘肃', '12'],
 ['黑龙江', '10'],
 ['辽宁', '9'],
 ['吉林', '7'],
 ['江苏', '5'],
 ['重庆', '2'],
 ['安徽', '1'],
 ['湖北', '0'],
 ['河北', '0'],
 ['新疆', '0'],
 ['江西', '0'],
 ['山西', '0'],
 ['海南', '0'],
 ['贵州', '0'],
 ['宁夏', '0'],
 ['澳门', '0'],
 ['青海', '0'],
 ['西藏', '0']]

In [26]:
c = (
    Map(init_opts=opts.InitOpts(width="1000px", height="600px"))  # 初始化地图大小
    .set_global_opts(
        title_opts=opts.TitleOpts(title="2021年12月25日疫情图"),  # 配置标题
        visualmap_opts=opts.VisualMapOpts(
            type_="scatter"  # 散点类型
        )
    )
    .add("now_exists", data, maptype="china")  # 将list传入，地图类型为中国地图
    .render("Map1.html")
)
print('请打开Map1.html')

请打开Map1.html


In [31]:
from pyecharts.globals import ThemeType  # 引入主题
c = (
    Map(init_opts=opts.InitOpts(width="1000px",
        height="600px", theme=ThemeType.ESSOS))  # 可切换主题
    .set_global_opts(
        title_opts=opts.TitleOpts(title="2021年12月25日疫情图"),
        visualmap_opts=opts.VisualMapOpts(
            min_=0,
            max_=600,
            range_text=['人数', ''],  # 分区间
            is_piecewise=True,  # 定义图例为分段型，默认为连续的图例
            pos_top="middle",  # 分段位置
            pos_left="left",
            orient="vertical",
            split_number=10  # 分成10个区间
        )
    )
    .add("now_exists", data, maptype="china")
    .render("Map2.html")
)
print('请打开Map2.html')

请打开Map2.html
